# OHCL of 3 bars - Strategy - Backtesting 

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')
from backtesting._plotting import set_bokeh_output
set_bokeh_output(notebook=False)

/home/zuongthao/PycharmProjects/algo-stock/venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [5]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2024-11-25 14:25:00,1298.0,1298.3,1296.7,1297.0,5524
2024-11-25 14:30:00,1297.1,1297.1,1297.1,1297.1,161
2024-11-25 14:45:00,1298.4,1298.4,1298.4,1298.4,5627


In [6]:
def get_signal(r):
    signal = ''
    if r['ohcl_s2'] > r['ohcl_s1'] > r['ohcl']:
        # ohcl giam dan
        signal = 'short'
    elif r['ohcl_s2'] < r['ohcl_s1'] < r['ohcl']:
        # ohcl tang dan
        signal = 'long'
    return signal

In [7]:
def prepare_data(data):
    data = data[data.High != data.Low]
    data['max_5'] = data['High'].rolling(5).max()
    data['min_5'] = data['Low'].rolling(5).min()
    data['ohcl'] = data.apply(lambda x: (x["Close"] - x["Open"]) / (x["High"] - x["Low"]), axis=1)
    data['ohcl_s1'] = data['ohcl'].shift(1)
    data['ohcl_s2'] = data['ohcl'].shift(2)
    data['signal'] = data.apply(lambda r: get_signal(r), axis=1)
    return data

In [8]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [9]:
prepared_data

,Open,High,Low,Close,Volume,max_5,min_5,ohcl,ohcl_s1,ohcl_s2,signal
Date,,,,,,,,,,,
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939,900.2,896.5,-0.142857,-0.615385,0.000000,
2020-11-02 09:25:00,898.3,898.6,897.8,898.5,1607,900.2,896.5,0.250000,-0.142857,-0.615385,long
2020-11-02 09:30:00,898.5,899.4,898.3,899.0,1623,900.0,896.5,0.454545,0.250000,-0.142857,long
2020-11-02 09:35:00,899.1,899.8,898.7,898.9,2180,899.8,896.5,-0.181818,0.454545,0.250000,
2020-11-02 09:40:00,899.0,899.4,898.7,898.8,1190,899.8,896.5,-0.285714,-0.181818,0.454545,short
...,...,...,...,...,...,...,...,...,...,...,...
2024-11-25 14:15:00,1296.7,1298.7,1296.2,1297.5,6636,1300.9,1295.9,0.320000,0.636364,-0.928571,
2024-11-25 14:20:00,1297.1,1298.3,1296.6,1298.0,4399,1299.9,1295.9,0.529412,0.320000,0.636364,
2024-11-25 14:25:00,1298.0,1298.3,1296.7,1297.0,5524,1298.7,1295.9,-0.625000,0.529412,0.320000,


In [10]:
prepared_data[prepared_data.signal != ''].tail(20)

,Open,High,Low,Close,Volume,max_5,min_5,ohcl,ohcl_s1,ohcl_s2,signal
Date,,,,,,,,,,,
2024-11-22 13:20:00,1297.4,1301.6,1297.4,1300.8,11538,1301.6,1295.9,0.809524,0.461538,0.000000,long
2024-11-22 13:35:00,1298.3,1299.0,1298.2,1299.0,3647,1301.8,1296.1,0.875000,-0.500000,-0.590909,long
2024-11-22 13:45:00,1300.0,1300.0,1294.2,1295.0,10516,1301.8,1294.2,-0.862069,0.666667,0.875000,short
2024-11-22 14:00:00,1294.7,1294.7,1291.3,1292.3,11661,1300.2,1291.3,-0.705882,-0.588235,0.409091,short
2024-11-22 14:15:00,1295.7,1296.1,1294.1,1294.7,5895,1296.6,1291.3,-0.500000,0.590909,0.612903,short
2024-11-22 14:25:00,1294.5,1296.4,1294.4,1294.6,7004,1296.6,1292.2,0.050000,-0.148148,-0.500000,long
2024-11-25 09:00:00,1298.6,1300.0,1298.4,1298.8,5736,1300.0,1292.2,0.125000,0.050000,-0.148148,long
2024-11-25 09:10:00,1298.8,1298.9,1297.4,1297.8,2875,1300.0,1292.2,-0.666667,0.111111,0.125000,short
2024-11-25 09:30:00,1297.3,1298.0,1297.1,1297.8,2069,1298.9,1296.2,0.555556,0.428571,-0.650000,long


In [11]:
class MainStrategy(Strategy):
    max_sl = 3.1
    trailing_sl = 4.5
    tp_step = 12
    def init(self):
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
                return
                
        if current_time.hour == 14 and current_time.minute >= 30:
            return

        close_price = self.data.Close[-1]
        if self.position.is_long:
            max_5 = self.data.max_5[-1]
            if close_price < max_5 - self.trailing_sl:
                self.position.close()
        elif self.position.is_short:
            min_5 = self.data.min_5[-1]
            if close_price > min_5 + self.trailing_sl:
                self.position.close()
 
        if self.position:
            return  
        signal = self.data.signal[-1]
        if signal == 'long':
            buy_price = close_price
            sl = buy_price - self.max_sl
            tp = buy_price + self.tp_step
            self.buy(size=1, sl=sl, tp=tp)
        elif signal == 'short':
            sell_price = close_price
            sl = sell_price + self.max_sl
            tp = sell_price - self.tp_step
            self.sell(size=1, sl=sl, tp=tp)

In [12]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [13]:
stats

Start                     2020-11-02 09:20:00
End                       2024-11-26 09:05:00
Duration                   1484 days 23:45:00
Exposure Time [%]                   81.832604
Equity Final [$]                  10240.49435
Equity Peak [$]                   10618.02796
Return [%]                           2.404944
Buy & Hold Return [%]               44.533511
Return (Ann.) [%]                    0.574004
Volatility (Ann.) [%]                1.650252
Sharpe Ratio                         0.347828
Sortino Ratio                        0.527179
Calmar Ratio                         0.144534
Max. Drawdown [%]                   -3.971406
Avg. Drawdown [%]                   -0.108479
Max. Drawdown Duration      712 days 20:00:00
Avg. Drawdown Duration        7 days 10:36:00
# Trades                                 4649
Win Rate [%]                        33.598623
Best Trade [%]                        3.24559
Worst Trade [%]                     -1.565099
Avg. Trade [%]                    

In [14]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,1,2,41,898.76955,898.0,-0.76955,-0.000856,2020-11-02 09:30:00,2020-11-02 14:10:00,None,0 days 04:40:00
1,1,43,45,899.26970,906.7,7.43030,0.008263,2020-11-02 14:20:00,2020-11-02 14:30:00,None,0 days 00:10:00
2,1,53,86,908.27240,904.9,-3.37240,-0.003713,2020-11-03 09:35:00,2020-11-03 13:45:00,None,0 days 04:10:00
3,1,89,95,906.37183,909.0,2.62817,0.002900,2020-11-03 14:00:00,2020-11-04 09:00:00,None,0 days 19:00:00
4,-1,100,110,907.82757,911.2,-3.37243,-0.003715,2020-11-04 09:25:00,2020-11-04 10:15:00,None,0 days 00:50:00
...,...,...,...,...,...,...,...,...,...,...,...
4644,1,49725,49739,1297.38910,1294.8,-2.58910,-0.001996,2024-11-22 11:15:00,2024-11-22 13:50:00,None,0 days 02:35:00
4645,-1,49742,49742,1292.11225,1295.4,-3.28775,-0.002544,2024-11-22 14:05:00,2024-11-22 14:05:00,None,0 days 00:00:00
4646,-1,49745,49747,1294.31159,1298.6,-4.28841,-0.003313,2024-11-22 14:20:00,2024-11-25 09:00:00,None,2 days 18:40:00
4647,1,49748,49756,1299.18964,1295.7,-3.48964,-0.002686,2024-11-25 09:05:00,2024-11-25 09:45:00,None,0 days 00:40:00
